### For data produced with DL1DH v0.7.4 (DL1_ML2)

In [ ]:
import tables
import numpy as np
import matplotlib.pyplot as plt
from ctapipe.instrument.camera import CameraGeometry
%matplotlib inline

In [ ]:
f = tables.open_file("/data2/deeplearning/datasets/DL1_ML2/Paranal/gamma-diffuse/South/train/gamma_20deg_180deg_runs1-1576___cta-prod3-demo_desert-2150m-Paranal-baseline_cone10.h5", "r")

### Provenance information stored as attributes

* Versions of involved code
* Corsika and simtel main parameters
* Simtel filenames

In [ ]:
f.root._v_attrs

### Flat file structure: everything else is stored as tables

* Three tables containing:
    + Array information
    + Telescope type information
    + Event information
* Plus one table per telescope containing:
    + 2x 1D arrays with calibrated pixels data per image (charge, arrival time)

In [ ]:
for node in f:
    print(node)

### Array information table

In [ ]:
f.root.Array_Information

In [ ]:
tel_types=['LST_LSTCam','MST_FlashCam','SST1M_DigiCam']

arr_table=f.root.Array_Information

fig, ax = plt.subplots(figsize=(10,10))

for tel_type in tel_types:
    tel_x = [x['x'] for x in arr_table.iterrows() if x['type'] == tel_type.encode('ascii')]
    tel_y = [x['y'] for x in arr_table.iterrows() if x['type'] == tel_type.encode('ascii')]
    plt.scatter(tel_x, tel_y, label=tel_type)
    
ax.legend()
ax.grid()
plt.show()

### Telescope information table

In [ ]:
f.root.Telescope_Type_Information

### Event information table

In [ ]:
f.root.Events

### Example 1: extract some infos from a given event

In [ ]:
event_index = 178
my_event = f.root.Events[event_index]
print('Event number: {}'.format(my_event['event_id']))
print('Energy: {} TeV'.format(my_event['mc_energy']))
print('x: {} m, y: {} m'.format(my_event['core_x'],my_event['core_y']))
print('Alt: {} rad'.format(my_event['alt']))
print('Az: {} rad'.format(my_event['az']))
tel_types = ['LST_LSTCam','MST_FlashCam','SST1M_DigiCam']
for tel_type in tel_types:
    tel_indices = '{}_indices'.format(tel_type)
    print('{} = {}'.format(tel_indices,my_event[tel_indices]))

#### Example 2: plot all charge and arrival time images for a given event

In [ ]:
tel_types = ['LST_LSTCam','MST_FlashCam','SST1M_DigiCam']

tel_type_size={'LST_LSTCam':15,'MST_FlashCam':15,'SST1M_DigiCam':15} #Size of the scatter plot point, for visualization purposes only

tel_table=f.root.Telescope_Type_Information

saveplots = False
combine = False

for tel_type in tel_types:
#    positions = [x['pixel_positions'] for x in tel_table.iterrows() if x['type'] == tel_type.encode('ascii')][0]
    camera_geometry = CameraGeometry.from_name(tel_type.split("_")[1])
    pos_x=camera_geometry.pix_x.value
    pos_y=camera_geometry.pix_y.value
    numpix = [x['num_pixels'] for x in tel_table.iterrows() if x['type'] == tel_type.encode('ascii')][0]
    exec('my_images = f.root.{}'.format(tel_type))
    my_indices = my_event['{}_indices'.format(tel_type)]
    img_charge_s = np.zeros(numpix)
    img_time_s = np.zeros(numpix)
    print('type: {} pixels: {}'.format(tel_type,len(img_charge_s)))
    for img_index in my_indices:
        if img_index > 0:
            img_charge = my_images[img_index]['charge']
            img_charge_s += img_charge
            img_time = my_images[img_index]['peakpos']
            img_time_s += img_time
            
            plt.figure(figsize=(15,5))
            plt.subplot(1, 2, 1)
            plt.scatter(pos_x, pos_y, c=img_charge, s=tel_type_size[tel_type])
            plt.axis('square')
            plt.colorbar()
            plt.subplot(1, 2, 2)
            plt.scatter(pos_x, pos_y, c=img_time, s=tel_type_size[tel_type])
            plt.axis('square')
            plt.colorbar()
            if saveplots:
                plt.savefig('{}_{}_{}.png'.format(event_index,tel_type,tel_ids[list(my_indices).index(img_index)]),\
                            dpi = 300, bbox_inches='tight',transparent=True)
            plt.show()
            
    if combine:
        plt.figure(figsize=(15,5))
        plt.subplot(1, 2, 1)
        plt.scatter(pos_x[:len(img_charge_s)], pos_y[:len(img_charge_s)], c=img_charge_s, s=tel_type_size[tel_type])
        plt.axis('square')
        plt.colorbar()
        plt.subplot(1, 2, 2)
        plt.scatter(pos_x[:len(img_time_s)], pos_y[:len(img_time_s)], c=img_time_s, s=tel_type_size[tel_type])
        plt.axis('square')
        plt.colorbar()
        if saveplots:
            plt.savefig('{}_{}.png'.format(event_index,tel_type),\
                        dpi = 300, bbox_inches='tight',transparent=True)
        plt.show()